In [14]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import requests
from src import insertar_datos_automatico, conexion_postgres, convertir_fechaeventos, combinar
pd.set_option('display.max_columns', None)
import psycopg2

In [15]:
df_jugadores = pd.read_csv("data/jugadores_info_normalizado.csv")

In [16]:
df_jugadores.columns

Index(['playerID', 'playerImage', 'playerName', 'playerFullName', 'birthplace',
       'dateOfBirth', 'dateOfDeath', 'playerShirtNumber', 'birthplaceCountry',
       'birthplaceCountryImage', 'age', 'height', 'foot', 'internationalTeam',
       'internationalTeamImage', 'internationalTeamStatus',
       'internationalGames', 'internationalGoals', 'internationalTeamShortTag',
       'internationalShirtNumber', 'internationalWmMember',
       'internationalValueRank', 'country', 'countrynameEN', 'countryImage',
       'countryShortName', 'secondCountry', 'secondCountryImage', 'league',
       'leaguenameEN', 'leagueLogo', 'clubImage', 'club', 'clubnameEN',
       'clubID', 'loan', 'contractExpiryDate', 'agent', 'agentId',
       'agentVerificationStatus', 'agentVerificationDate', 'outfitter',
       'positionGroup', 'playerMainPosition', 'playerSecondPosition',
       'playerThirdPosition', 'marketValue', 'marketValueCurrency',
       'marketValueNumeral', 'marketValueLastChange', 'relat

In [17]:
df_jugadores_columnasok = df_jugadores[['playerID', 'playerName', 'playerFullName', 'birthplace',
       'dateOfBirth', 'birthplaceCountry', 'age', 'height', 'foot', 'internationalTeam',
       'internationalGames', 'internationalGoals', 'country', 'secondCountry', 'club',
       'clubID', 'playerMainPosition', 'playerSecondPosition',
       'playerThirdPosition', 'matches_played', 'starts', 'min', 'gls',
       'ast', 'non_pen_gls', 'yellow_cards', 'red_cards',
       'per90_gls', 'per90_ast', 'per90_non_pen_gls', 'subs',]]

In [18]:
df_jugadores_columnasok.rename(columns = {
    'playerName': 'Nombre_Jugador',
    'playerFullName': 'Nombre_Completo_Jugador',
    'birthplace': 'Lugar_Nacimiento',
    'dateOfBirth': 'Fecha_Nacimiento',
    'birthplaceCountry': 'Pais_Nacimiento',
    'age': 'Edad',
    'height': 'Altura',
    'foot': 'Pie_Dominante',
    'internationalTeam': 'Equipo_Internacional',
    'internationalGames': 'Partidos_Internacionales',
    'internationalGoals': 'Goles_Internacionales',
    'country': 'Pais',
    'secondCountry': 'Segundo_Pais',
    'club': 'Club',
    'playerMainPosition': 'Posicion_Principal',
    'playerSecondPosition': 'Posicion_Secundaria',
    'playerThirdPosition': 'Posicion_Terciaria',
    'matches_played' : 'Partidos_jugados', 
    'starts': 'Titularidades', 
    'min': 'Minutos', 
    'gls': 'Goles',
    'ast': 'Asistencias',
    'non_pen_gls': 'Goles_sin_penaltis',
    'yellow_cards': 'Amarillas',
    'red_cards': 'Rojas',
    'per90_gls': 'Goles_por_90',
    'per90_ast': 'Asistencias_por_90',
    'per90_non_pen_gls': 'Goles_sin_penaltis_por_90',
    'subs': 'Suplencias'
}, inplace = True)

C:\Users\pepel\AppData\Local\Temp\ipykernel_824\3780516607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jugadores_columnasok.rename(columns = {


In [19]:
df_jugadores_columnasok = df_jugadores_columnasok.dropna(subset=["playerID"])
df_jugadores_columnasok["Altura"].replace("indeterminado", np.nan, inplace=True)
df_jugadores_columnasok["Altura"] = df_jugadores_columnasok["Altura"].str.replace(",", ".").astype(float)

C:\Users\pepel\AppData\Local\Temp\ipykernel_824\2153508300.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_jugadores_columnasok["Altura"].replace("indeterminado", np.nan, inplace=True)


In [20]:
df_jugadores_columnasok["Fecha_Nacimiento"] = df_jugadores_columnasok["Fecha_Nacimiento"] = pd.to_datetime(df_jugadores_columnasok["Fecha_Nacimiento"])

C:\Users\pepel\AppData\Local\Temp\ipykernel_824\3186894547.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_jugadores_columnasok["Fecha_Nacimiento"] = df_jugadores_columnasok["Fecha_Nacimiento"] = pd.to_datetime(df_jugadores_columnasok["Fecha_Nacimiento"])


In [21]:
max(df_jugadores_columnasok["playerID"])

220761.0

In [23]:
df_jugadores_columnasok.columns

Index(['playerID', 'Nombre_Jugador', 'Nombre_Completo_Jugador',
       'Lugar_Nacimiento', 'Fecha_Nacimiento', 'Pais_Nacimiento', 'Edad',
       'Altura', 'Pie_Dominante', 'Equipo_Internacional',
       'Partidos_Internacionales', 'Goles_Internacionales', 'Pais',
       'Segundo_Pais', 'Club', 'clubID', 'Posicion_Principal',
       'Posicion_Secundaria', 'Posicion_Terciaria', 'Partidos_jugados',
       'Titularidades', 'Minutos', 'Goles', 'Asistencias',
       'Goles_sin_penaltis', 'Amarillas', 'Rojas', 'Goles_por_90',
       'Asistencias_por_90', 'Goles_sin_penaltis_por_90', 'Suplencias'],
      dtype='object')

In [24]:
df_jugadores_columnasok["playerID"] = df_jugadores_columnasok["playerID"].astype(int)
df_jugadores_columnasok["clubID"] = df_jugadores_columnasok["clubID"].astype(int)
df_jugadores_columnasok["Partidos_Internacionales"] = df_jugadores_columnasok["Partidos_Internacionales"].astype('Int64')
df_jugadores_columnasok["Goles_Internacionales"] = df_jugadores_columnasok["Goles_Internacionales"].astype('Int64')
df_jugadores_columnasok["Titularidades"] = df_jugadores_columnasok["Titularidades"].astype('Int64')
df_jugadores_columnasok["Goles"] = df_jugadores_columnasok["Goles"].astype('Int64')
df_jugadores_columnasok["Asistencias"] = df_jugadores_columnasok["Asistencias"].astype('Int64')
df_jugadores_columnasok["Amarillas"] = df_jugadores_columnasok["Amarillas"].astype('Int64')
df_jugadores_columnasok["Rojas"] = df_jugadores_columnasok["Rojas"].astype('Int64')
#df_jugadores_columnasok.replace({pd.NA: None}, inplace=True)

In [25]:
for columna in df_jugadores_columnasok.columns:
    if df_jugadores_columnasok[columna].dtype in ['int64', 'float64', int]:
        min_val = df_jugadores_columnasok[columna].min()
        max_val = df_jugadores_columnasok[columna].max()
        print(f"Columna {columna}: Min = {min_val}, Max = {max_val}, {type(max_val)}")

Columna playerID: Min = 3023, Max = 220761, <class 'numpy.int64'>
Columna Edad: Min = 0, Max = 49, <class 'numpy.int64'>
Columna Altura: Min = 1.64, Max = 1.96, <class 'numpy.float64'>
Columna clubID: Min = 207, Max = 1049, <class 'numpy.int64'>
Columna Partidos_jugados: Min = 1.0, Max = 38.0, <class 'numpy.float64'>
Columna Minutos: Min = 1.0, Max = 3420.0, <class 'numpy.float64'>
Columna Goles_sin_penaltis: Min = 0.0, Max = 37.0, <class 'numpy.float64'>
Columna Goles_por_90: Min = 0.0, Max = 1.45, <class 'numpy.float64'>
Columna Asistencias_por_90: Min = 0.0, Max = 2.57, <class 'numpy.float64'>
Columna Goles_sin_penaltis_por_90: Min = 0.0, Max = 1.45, <class 'numpy.float64'>
Columna Suplencias: Min = 0.0, Max = 23.0, <class 'numpy.float64'>


In [26]:
len(df_jugadores_columnasok.columns)

31

In [27]:
df_jugadores_columnasok[df_jugadores_columnasok["playerID"] == 4711]

,playerID,Nombre_Jugador,Nombre_Completo_Jugador,Lugar_Nacimiento,Fecha_Nacimiento,Pais_Nacimiento,Edad,Altura,Pie_Dominante,Equipo_Internacional,Partidos_Internacionales,Goles_Internacionales,Pais,Segundo_Pais,Club,clubID,Posicion_Principal,Posicion_Secundaria,Posicion_Terciaria,Partidos_jugados,Titularidades,Minutos,Goles,Asistencias,Goles_sin_penaltis,Amarillas,Rojas,Goles_por_90,Asistencias_por_90,Goles_sin_penaltis_por_90,Suplencias
4140,4711,Pablo Hernández,Pablo Hernández Domínguez,Castellón,1985-04-11,España,40,1.73,Derecho,España,4,1,España,NaN,Rayo Vallecano,360,Extremo derecho,Mediocentro ofensivo,Extremo izquierdo,33.0,28,2304.0,2,4,2.0,15,1,0.08,0.16,0.08,5.0
4141,4711,Pablo Hernández,Pablo Hernández Domínguez,Castellón,1985-04-11,España,40,1.73,Derecho,España,4,1,España,NaN,Rayo Vallecano,360,Extremo derecho,Mediocentro ofensivo,Extremo izquierdo,27.0,24,1908.0,3,4,3.0,3,0,0.14,0.19,0.14,3.0


In [28]:
df_jugadores_columnasok[df_jugadores_columnasok["playerID"].duplicated()]

,playerID,Nombre_Jugador,Nombre_Completo_Jugador,Lugar_Nacimiento,Fecha_Nacimiento,Pais_Nacimiento,Edad,Altura,Pie_Dominante,Equipo_Internacional,Partidos_Internacionales,Goles_Internacionales,Pais,Segundo_Pais,Club,clubID,Posicion_Principal,Posicion_Secundaria,Posicion_Terciaria,Partidos_jugados,Titularidades,Minutos,Goles,Asistencias,Goles_sin_penaltis,Amarillas,Rojas,Goles_por_90,Asistencias_por_90,Goles_sin_penaltis_por_90,Suplencias
371,5186,Denis Cheryshev,Denis Cheryshev,Nizhniy Novgorod,1990-12-26,UdSSR,34,1.79,Izquierdo,Rusia,33,12,Rusia,España,Real Madrid,220,Extremo izquierdo,Extremo derecho,Mediapunta,7.0,6,478.0,3,0,3.0,2,0,0.56,0.00,0.56,1.0
669,6391,Raúl García,Raúl García Escudero,Zizur Mayor,1986-07-11,España,38,1.84,Derecho,España,2,0,España,NaN,Atlético Madrid,212,Mediocentro ofensivo,Mediocentro,Delantero centro,30.0,27,2370.0,7,6,7.0,10,0,0.27,0.23,0.27,3.0
671,19668,Augusto Fernández,Augusto Matías Fernández,Pergamino,1986-04-10,Argentina,39,1.77,Derecho,Argentina,16,1,Argentina,Italia,Celta Vigo,209,Mediocentro,Pivote,Interior derecho,15.0,15,1337.0,1,1,1.0,6,0,0.07,0.07,0.07,0.0
673,26780,Guilherme Siqueira,Guilherme Magdalena Siqueira,Florianópolis,1986-04-28,Brasil,38,1.82,Izquierdo,NaN,<NA>,<NA>,Brasil,Italia,Atlético Madrid,212,Lateral izquierdo,NaN,NaN,14.0,14,1093.0,0,0,0.0,1,0,0.00,0.00,0.00,0.0
2243,27480,Lucas Orbán,Lucas Alfonso Orbán Alegre,Buenos Aires,1989-02-03,Argentina,36,1.79,Izquierdo,Argentina,2,0,Argentina,España,Valencia,207,Defensa central,Lateral izquierdo,NaN,3.0,3,270.0,0,0,0.0,2,0,0.00,0.00,0.00,0.0
3708,25707,Fran Rico,Francisco Manuel Rico Castro,Portonovo,1987-08-03,España,37,1.78,Derecho,NaN,<NA>,<NA>,España,NaN,Granada,1049,Mediocentro,Pivote,NaN,8.0,1,158.0,0,2,0.0,0,0,0.00,1.14,0.00,7.0
3977,13107,Sergio Álvarez,Sergio Álvarez Díaz,Avilés,1992-01-23,España,33,1.83,Derecho,España Sub-19,3,0,España,NaN,Sporting Gijón,1041,Pivote,Mediocentro,Defensa central,27.0,26,2185.0,2,2,2.0,9,1,0.08,0.08,0.08,1.0
4141,4711,Pablo Hernández,Pablo Hernández Domínguez,Castellón,1985-04-11,España,40,1.73,Derecho,España,4,1,España,NaN,Rayo Vallecano,360,Extremo derecho,Mediocentro ofensivo,Extremo izquierdo,27.0,24,1908.0,3,4,3.0,3,0,0.14,0.19,0.14,3.0


In [29]:
df_jugadores_columnasok[df_jugadores_columnasok["playerID"] == 27480]

,playerID,Nombre_Jugador,Nombre_Completo_Jugador,Lugar_Nacimiento,Fecha_Nacimiento,Pais_Nacimiento,Edad,Altura,Pie_Dominante,Equipo_Internacional,Partidos_Internacionales,Goles_Internacionales,Pais,Segundo_Pais,Club,clubID,Posicion_Principal,Posicion_Secundaria,Posicion_Terciaria,Partidos_jugados,Titularidades,Minutos,Goles,Asistencias,Goles_sin_penaltis,Amarillas,Rojas,Goles_por_90,Asistencias_por_90,Goles_sin_penaltis_por_90,Suplencias
2242,27480,Lucas Orbán,Lucas Alfonso Orbán Alegre,Buenos Aires,1989-02-03,Argentina,36,1.79,Izquierdo,Argentina,2,0,Argentina,España,Valencia,207,Defensa central,Lateral izquierdo,NaN,7.0,6,521.0,0,0,0.0,4,1,0.0,0.0,0.0,1.0
2243,27480,Lucas Orbán,Lucas Alfonso Orbán Alegre,Buenos Aires,1989-02-03,Argentina,36,1.79,Izquierdo,Argentina,2,0,Argentina,España,Valencia,207,Defensa central,Lateral izquierdo,NaN,3.0,3,270.0,0,0,0.0,2,0,0.0,0.0,0.0,0.0


In [30]:
indices = df_jugadores_columnasok[df_jugadores_columnasok["playerID"].duplicated()].index

In [31]:
# --- UNIFICAR FILAS DUPLICADAS ---
for x in indices:
    fila_actual = df_jugadores_columnasok.loc[x]
    fila_anterior = df_jugadores_columnasok.loc[x - 1]

    # Aseguramos tipos escalar combinando columna por columna
    fila_unificada = pd.Series(
        {col: combinar(fila_actual[col], fila_anterior[col]) for col in df_jugadores_columnasok.columns}
    )

    # Reemplazar la fila anterior con la fila unificada
    df_jugadores_columnasok.loc[x - 1] = fila_unificada

# --- ELIMINAR LAS FILAS DUPLICADAS (las actuales) ---
df_jugadores_columnasok = df_jugadores_columnasok.drop(index=indices).reset_index(drop=True)


In [32]:
df_jugadores_columnasok.dtypes

playerID                              int64
Nombre_Jugador                       object
Nombre_Completo_Jugador              object
Lugar_Nacimiento                     object
Fecha_Nacimiento             datetime64[ns]
Pais_Nacimiento                      object
Edad                                  int64
Altura                              float64
Pie_Dominante                        object
Equipo_Internacional                 object
Partidos_Internacionales              Int64
Goles_Internacionales                 Int64
Pais                                 object
Segundo_Pais                         object
Club                                 object
clubID                                int64
Posicion_Principal                   object
Posicion_Secundaria                  object
Posicion_Terciaria                   object
Partidos_jugados                    float64
Titularidades                         Int64
Minutos                             float64
Goles                           

In [33]:
df_jugadores_columnasok = df_jugadores_columnasok.astype(object).where(pd.notna(df_jugadores_columnasok), None)

In [38]:
load_dotenv()
dbname = os.getenv("dbname")
user = os.getenv("user")
password = os.getenv("password")
(conn,cur) = conexion_postgres(dbname, user, password)
insertar_datos_automatico(df_jugadores_columnasok,"jugadores", cur, conn)

('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)


In [35]:
conn.rollback()

In [36]:
df_jugadores_columnasok.to_csv("data/Tabla_Jugadores.csv", index=False)